# Analyzing ACS PUMS Data

In [1]:
from pathlib import Path
import pandas as pd
from datetime import datetime as dt
from acs_download import tree

today = dt.today().strftime("%d-%b-%y")

C:\Users\sanchez\AppData\Local\Continuum\anaconda3\envs\census\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
RAW_DATA = Path("../data/raw/")
INTERIM_DATA = Path("../data/interim/")

In [3]:
tree(INTERIM_DATA)

+ ..\data\interim
    + ACS_2017
        + oh
            + ACS2017_PUMS_README.pdf
            + psam_p39.csv


In [4]:
data = pd.read_csv(INTERIM_DATA / 'ACS_2017' / 'oh' / 'psam_p39.csv')

#### There is a **LOT** of information in these Public Use Micro Sample data files.

In [5]:
data.shape

(118842, 286)

In [6]:
data.head()

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
0,P,2017000000064,3,1,3000,2,39,1011189,55,79,...,46,92,14,51,18,54,97,54,98,85
1,P,2017000000064,3,2,3000,2,39,1011189,69,79,...,53,113,19,65,19,66,154,69,130,95
2,P,2017000000143,3,1,4110,2,39,1011189,83,28,...,101,26,113,37,26,79,86,153,132,115
3,P,2017000000147,3,1,2900,2,39,1011189,159,78,...,160,228,57,175,187,301,186,246,244,153
4,P,2017000000147,3,2,2900,2,39,1011189,169,52,...,228,207,47,129,220,295,189,264,198,213


If you go to the census bureau's [website](https://www.census.gov/programs-surveys/acs/technical-documentation/pums/documentation.html) you can find the [2017 ACS 1-year PUMS data dictionary](https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMS_Data_Dictionary_2017.pdf?#).

From there you can choose your variables of interest. In this case, we'll find if recent immigrants are more or less likely than native Californian's to have a college degree.

The variables we're interested in are therefore, 
1. `AGEP` - "Age"
2. `SCHL` - "Educational Attainment"
3. `CIT`  - "Citizenship Status"
4. `YOEP` - "Year of Entry"
5. `PWGTP` - "Person's weight"

In [7]:
variables_of_interest = ['AGEP', 'SCHL', 'CIT', 'YOEP', 'PWGTP']

working_data = data[variables_of_interest].copy()

In [8]:
print(working_data.shape)
working_data.head()

(118842, 5)


,AGEP,SCHL,CIT,YOEP,PWGTP
0,79,23.0,1,NaN,55
1,79,23.0,1,NaN,69
2,28,12.0,1,NaN,83
3,78,16.0,1,NaN,159
4,52,16.0,1,NaN,169


This subset of data still has 380,000 rows. However, we're interested in a very selective group of people for our analysis.

We are looking at _recent immigrants_, which we arbitrarily defined as _those arrived within the past 5 years (of taking the survey)_. So `YOEP` **must** be 2012 or higher. We're also looking at _working age adults_ who would have had the time to get a college degree if they wanted, so we are dropping anyone below 25 and 65 or more (traditional age of retirement in the U.S.).

Note: `YOEP` is `NULL` or `NaN` for U.S. Born Californian's, so filtering by `YOEP` would drop all US-born Californians from our dataset. To fix this, we _temporarily_ fill those Null values with a really high number for the filtering. Doing it this way means you don't actually change the dataframe, which we don't want to do, but you can filter using this _modified_ version of it (with null values filled).

In [9]:
yoep_filter = working_data['YOEP'].fillna(9999) >= 2012
agep_filter = (working_data['AGEP'] >= 25) & (working_data['AGEP'] <= 64)

In [10]:
working_data[yoep_filter & agep_filter].shape

(57844, 5)

This reduces our dataset's size significantly.

In [11]:
working_data = working_data[yoep_filter & agep_filter].copy()

Now we need to add some value labels to our `SCHL` and `CIT` variables. 

We do this using the data dictionary we looked at earlier.
We find that:
* `CIT` - Citizenship status
    1. Born in the U.S.
    2. Born in Puerto Rico, Guam, the U.S. Virgin Islands, or the Northern Marianas.
    3. Born abroad of American parent(s).
    4. U.S. citizen by naturalization.
    5. Not a citizen of the U.S.


* `SCHL` - Educational Attainment
    1. No schooling completed
    2. Nursery school, preschool
    3. Kindergarten
    4. Grade 1
    5. Grade 2
    6. Grade 3
    7. Grade 4
    8. Grade 5
    9. Grade 6
    10. Grade 7
    11. Grade 8
    12. Grade 9
    13. Grade 10
    14. Grade 11
    15. 12th grade - no diploma
    16. Regular high school diploma
    17. GED or alternative credential
    18. Some college, but less than 1 year
    19. 1 or more years of college credit, no degree
    20. Associate's degree
    21. Bachelor's degree
    22. Master's degree
    23. Professional degree beyond a bachelor's degree
    24. Doctorate degree


The way to _map_ these value labels to our `pandas` dataframe is by using a `dict`

In [12]:
value_labels = {}

value_labels['CIT'] = {
    1: "Born in the U.S.",
    2: "Born in Puerto Rico, Guam, the U.S. Virgin Islands, or the Northern Marianas.",
    3: "Born abroad of American parent(s).",
    4: "U.S. citizen by naturalization.",
    5: "Not a citizen of the U.S.",
}

value_labels['SCHL'] = {
    1: "No schooling completed",
    2: "Nursery school, preschool",
    3: "Kindergarten",
    4: "Grade 1",
    5: "Grade 2",
    6: "Grade 3",
    7: "Grade 4",
    8: "Grade 5",
    9: "Grade 6",
    10: "Grade 7",
    11: "Grade 8",
    12: "Grade 9",
    13: "Grade 10",
    14: "Grade 11",
    15: "12th grade - no diploma",
    16: "Regular high school diploma",
    17: "GED or alternative credential",
    18: "Some college, but less than 1 year",
    19: "1 or more years of college credit, no degree",
    20: "Associate's degree",
    21: "Bachelor's degree",
    22: "Master's degree",
    23: "Professional degree beyond a bachelor's degree",
    24: "Doctorate degree",
}

First, we map these value labels to our dataframe. Since we have each of the value labels' dictionaries within our `value_labels` dictionary, we can iterate over the columns and _map_ our labels **if** we have them store in our `value_labels` dictionary.

In [13]:
for column in working_data.columns:
    if column in value_labels.keys():
        working_data[column] = working_data[column].map(value_labels[column])

¡Listo!

In [14]:
working_data.head()

,AGEP,SCHL,CIT,YOEP,PWGTP
2,28,Grade 9,Born in the U.S.,NaN,83
4,52,Regular high school diploma,Born in the U.S.,NaN,169
5,26,Regular high school diploma,Born in the U.S.,NaN,190
6,54,Regular high school diploma,Born in the U.S.,NaN,174
7,54,Regular high school diploma,Born in the U.S.,NaN,24


However, previously our data had some sort of _order_ because it was coded in numbers. We know 3 is greater than 2. This doesn't mean much in our `CIT` column (_Born abroad of American parent(s)_ isn't **greater** than _Born in Puerto Rico, Guam, US Virgin Islands, or Northern Marianas_). In our `SCHL` column, though, it would make sense to keep the order of our values. In educational attainment, we know that **Grade 8** comes after **Grade 7**, so it is a _greater_ amount of education.

For these types of situations there is the `Categorical` dtype in pandas.

In [15]:
pd.Categorical?

Init signature:
pd.Categorical(
    values,
    categories=None,
    ordered=None,
    dtype=None,
    fastpath=False,
)
Docstring:     
Represents a categorical variable in classic R / S-plus fashion

`Categoricals` can only take on only a limited, and usually fixed, number
of possible values (`categories`). In contrast to statistical categorical
variables, a `Categorical` might have an order, but numerical operations
(additions, divisions, ...) are not possible.

All values of the `Categorical` are either in `categories` or `np.nan`.
Assigning values outside of `categories` will raise a `ValueError`. Order
is defined by the order of the `categories`, not lexical order of the
values.

Parameters
----------
values : list-like
    The values of the categorical. If categories are given, values not in
    categories will be replaced with NaN.
categories : Index-like (unique), optional
    The unique categories for this categorical. If not given, the
    categories are assumed to be the un

We'll do both columns, `CIT` and `SCHL`, as eventually we'll be creating other groups from `CIT` (Native and Foreign-born), and comparing data with `Categoricals` is a little easier than comparing strings.

Note: since we're only doing 2 columns, we don't _need_ to iterate but for scalability purposes we'll keep the same workflow as we've had so far. 

In [16]:
for key in value_labels.keys():
    working_data[key] = pd.Categorical(
                            values = working_data[key],
                            categories = value_labels[key].values(),
                            ordered = True
                        )

In [17]:
working_data.head()

,AGEP,SCHL,CIT,YOEP,PWGTP
2,28,Grade 9,Born in the U.S.,NaN,83
4,52,Regular high school diploma,Born in the U.S.,NaN,169
5,26,Regular high school diploma,Born in the U.S.,NaN,190
6,54,Regular high school diploma,Born in the U.S.,NaN,174
7,54,Regular high school diploma,Born in the U.S.,NaN,24


The dataframe may look the same but <br>
1) it's memory size has been reduced drastically, categorical data takes less space than strings. <br>
2) you can now use logical operators `<` and `>`

for example,

In [18]:
example_filter = working_data['SCHL'] > 'Regular high school diploma'
working_data[example_filter].groupby('SCHL')[['PWGTP']].sum()

,PWGTP
SCHL,
No schooling completed,0
"Nursery school, preschool",0
Kindergarten,0
Grade 1,0
Grade 2,0
Grade 3,0
Grade 4,0
Grade 5,0
Grade 6,0


In [19]:
example_filter = working_data['SCHL'] <= 'Regular high school diploma'
working_data[example_filter].groupby('SCHL')[['PWGTP']].sum()

,PWGTP
SCHL,
No schooling completed,33920
"Nursery school, preschool",1644
Kindergarten,1159
Grade 1,293
Grade 2,3427
Grade 3,883
Grade 4,1178
Grade 5,3135
Grade 6,3674


Grouping by categorical data will still display the rest of the labels even if there is no data associated with them.

Using this logic we can create a new education variable with a smaller set of groups.
1. Not a high school graduate
2. High school graduate
3. Some college
4. Bachelor's degree
5. Graduate degree

First, we create our filters:

In [20]:
# one way to do it 
filter_not_hs = working_data['SCHL'] <= "12th grade - no diploma"
filter_hs = (working_data['SCHL'] == "Regular high school diploma") | (working_data['SCHL'] == 'GED or alternative credential')
filter_some_college = (working_data['SCHL'] >= 'Some college, but less than 1 year') & (working_data['SCHL'] <= "Associate's degree")
filter_bachelors = working_data['SCHL'] == "Bachelor's degree"
filter_grad_degree = working_data['SCHL'] >= "Master's degree"

We use the `.loc` accessor to modify values in our dataframe.

In [21]:
# not hs
working_data.loc[filter_not_hs, 'EDUC5'] = 'Not a high school graduate'

# hs
working_data.loc[filter_hs, 'EDUC5'] = 'High school graduate'

# some college
working_data.loc[filter_some_college, 'EDUC5'] = 'Some college'

# bachelor's
working_data.loc[filter_bachelors, 'EDUC5'] = "Bachelor's degree"

# graduate degree
working_data.loc[filter_grad_degree, 'EDUC5'] = 'Graduate degree'

Note: You could make this new column a categorical if you want to.

In [22]:
working_data["EDUC5"] = pd.Categorical(
                        values = working_data["EDUC5"],
                        categories = ['Not a high school graduate', 'High school graduate', 'Some college', "Bachelor's degree", 'Graduate degree'],
                        ordered = True
                    )

In [23]:
working_data.head()

,AGEP,SCHL,CIT,YOEP,PWGTP,EDUC5
2,28,Grade 9,Born in the U.S.,NaN,83,Not a high school graduate
4,52,Regular high school diploma,Born in the U.S.,NaN,169,High school graduate
5,26,Regular high school diploma,Born in the U.S.,NaN,190,High school graduate
6,54,Regular high school diploma,Born in the U.S.,NaN,174,High school graduate
7,54,Regular high school diploma,Born in the U.S.,NaN,24,High school graduate


Now we'll do the same with `CIT`

In [24]:
# filters
filter_native = working_data['CIT'] <= "Born abroad of American parent(s)."
filter_foreign_born = working_data['CIT'] > "Born abroad of American parent(s)."

In [25]:
# because this is a binary choice we could do the following instead
# natives
working_data.loc[filter_native, 'NATIVITY'] = 'Native'

# foreign-born
working_data.loc[~filter_native, 'NATIVITY'] = 'Foreign-born'

`~` == `not`

In [26]:
working_data.head()

,AGEP,SCHL,CIT,YOEP,PWGTP,EDUC5,NATIVITY
2,28,Grade 9,Born in the U.S.,NaN,83,Not a high school graduate,Native
4,52,Regular high school diploma,Born in the U.S.,NaN,169,High school graduate,Native
5,26,Regular high school diploma,Born in the U.S.,NaN,190,High school graduate,Native
6,54,Regular high school diploma,Born in the U.S.,NaN,174,High school graduate,Native
7,54,Regular high school diploma,Born in the U.S.,NaN,24,High school graduate,Native


### Now we have all we need!

In [27]:
data_table = working_data.groupby(['NATIVITY', 'EDUC5'])[['PWGTP']].sum()

data_table

PWGTP
NATIVITY     EDUC5                              
Foreign-born Not a high school graduate    10484
             High school graduate          14074
             Some college                   8978
             Bachelor's degree             23133
             Graduate degree               22255
Native       Not a high school graduate   455001
             High school graduate        1781382
             Some college                1773105
             Bachelor's degree           1046321
             Graduate degree              568953

In [28]:
# pivot tables are similar to groupby's but have the added feature of having a "totals" col and row
working_data.pivot_table(index = ['NATIVITY', 'EDUC5'], values = 'PWGTP', aggfunc = 'sum', margins = True, margins_name = 'Total')

PWGTP
NATIVITY     EDUC5                              
Foreign-born Not a high school graduate    10484
             High school graduate          14074
             Some college                   8978
             Bachelor's degree             23133
             Graduate degree               22255
Native       Not a high school graduate   455001
             High school graduate        1781382
             Some college                1773105
             Bachelor's degree           1046321
             Graduate degree              568953
Total                                    5703686

Now we have the actual number of people in each of these groups but what we want is to find the "% share" they make up of their total.

In other words, <br>
`total of Bachelor's degrees who are foreign-born / total of people who are foreign-born`

To do this, we need to go _row by row_ in each of our columns. Grab the total of either Foreign-born or Native, and then look at each of their rows (bachelor's degrees, graduates, hs, not hs) and calculate the share they make up of their group's total.

We do this by _applying_ a function to each of our dataframe's rows.

In [29]:
data_table.apply(lambda group: group/group.sum())

PWGTP
NATIVITY     EDUC5                               
Foreign-born Not a high school graduate  0.001838
             High school graduate        0.002468
             Some college                0.001574
             Bachelor's degree           0.004056
             Graduate degree             0.003902
Native       Not a high school graduate  0.079773
             High school graduate        0.312321
             Some college                0.310870
             Bachelor's degree           0.183446
             Graduate degree             0.099752

This doesn't seem right though. Right now, we're grabbing each row and dividing it from the total amount of people in our dataframe. What we want is our denominator to be the **total amount of people in our NATIVITY group**. So we need to `groupby` first.

In [30]:
shares = data_table.groupby("NATIVITY").apply(lambda group: group / group.sum())

shares

PWGTP
NATIVITY     EDUC5                               
Foreign-born Not a high school graduate  0.132837
             High school graduate        0.178323
             Some college                0.113755
             Bachelor's degree           0.293105
             Graduate degree             0.281980
Native       Not a high school graduate  0.080892
             High school graduate        0.316704
             Some college                0.315232
             Bachelor's degree           0.186020
             Graduate degree             0.101151

¡Listo!

We've found the share of Californian's in each of our educational attainment categories by Nativity. Next step is to visualize it. <br>
We're not going to do that in this notebook to keep our workflow clean and organized. Let's save this and move on to the next notebook.

In [31]:
shares.to_csv(INTERIM_DATA / f"educ_attainment_by_nativity-{today}.csv",)

Not everyone likes to use pandas dataframe's indexes like this and would rather have a dataframe that looks more like

In [32]:
shares.reset_index()

,NATIVITY,EDUC5,PWGTP
0,Foreign-born,Not a high school graduate,0.132837
1,Foreign-born,High school graduate,0.178323
2,Foreign-born,Some college,0.113755
3,Foreign-born,Bachelor's degree,0.293105
4,Foreign-born,Graduate degree,0.281980
5,Native,Not a high school graduate,0.080892
6,Native,High school graduate,0.316704
7,Native,Some college,0.315232
8,Native,Bachelor's degree,0.186020
9,Native,Graduate degree,0.101151


If you reset the index on a _grouped_ dataframe I would advise you to **not** save the index in your `.csv` file.

```python
no_index_shares = shares.reset_index()

no_index_shares.to_csv(INTERIM_DATA / f"educ_attainment_by_nativity-{today}.csv", index = False)
```